Libraries and Imports

In [ ]:
import json
import os
import sys

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.utils.data as data
from ncps.torch import LTC
from ncps.wirings import AutoNCP
from plyer import notification
from pytorch_lightning.loggers import CSVLogger

sys.path.append(os.path.abspath("funcs"))

from config_reading import read_configs
from timer_callback import TimingCallback

Opening configuration file

In [ ]:
configs = read_configs()
processing_configs = configs["processing"]
training_configs = configs["training"]

Defining Model Name

In [ ]:
model_name = f"r_{configs['model_name']}"

Setting the CUDA float32 precision.

In [ ]:
torch.set_float32_matmul_precision(training_configs["float_precision"])

Set the seed manually to ensure reproducibility

In [ ]:
seed = training_configs["seed"]

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
pl.seed_everything(seed, workers=True)

Reading the dataset.

In [ ]:
has_validation = processing_configs["validation_proportion"] > 0

In [ ]:
train_x = torch.load(os.path.join(processing_configs["save_path"], "tensor_train_x.pt"))
train_y = torch.load(os.path.join(processing_configs["save_path"], "tensor_train_y.pt"))
if has_validation:
	val_x = torch.load(os.path.join(processing_configs["save_path"], "tensor_val_x.pt"))
	val_y = torch.load(os.path.join(processing_configs["save_path"], "tensor_val_y.pt"))
test_x = torch.load(os.path.join(processing_configs["save_path"], "tensor_test_x.pt"))
test_y = torch.load(os.path.join(processing_configs["save_path"], "tensor_test_y.pt"))

Converting the labels to a label-positional list

In [ ]:
conversion_dict = {0: [1, 0, 0, 0, 0, 0], 1: [0, 1, 0, 0, 0, 0], 
                   2: [0, 0, 1, 0, 0, 0], 3: [0, 0, 0, 1, 0, 0], 
                   4: [0, 0, 0, 0, 1, 0], 5: [0, 0, 0, 0, 0, 1]}

def convert_to_label_positional(data):
  result = []
  for e in data:
    result.append(conversion_dict[e])

  return torch.tensor(np.array(result), dtype=torch.float32)

train_y = convert_to_label_positional(train_y.numpy())
if has_validation:
	val_y = convert_to_label_positional(val_y.numpy())
test_y = convert_to_label_positional(test_y.numpy())

Defining loaders, models phases and model configuration

In [ ]:
train_batch, val_batch, test_batch = training_configs["batch_sizes"]

if train_batch == "all":
  train_batch = train_x.shape[0]
if has_validation and val_batch == "all":
  val_batch = val_x.shape[0]
if test_batch == "all":
  test_batch = test_x.shape[0]

In [ ]:
train_dataloader = data.DataLoader(data.TensorDataset(train_x, train_y), shuffle=True, num_workers=16, persistent_workers=True, batch_size=train_batch)
if has_validation:
	val_dataloader = data.DataLoader(data.TensorDataset(val_x, val_y), num_workers=16, persistent_workers=True, batch_size=val_batch)
test_dataloader = data.DataLoader(data.TensorDataset(test_x, test_y), num_workers=16, persistent_workers=True, batch_size=test_batch)

In [ ]:
class SequenceLearner(pl.LightningModule):
  def __init__(self, model, lr):
    super().__init__()
    self.model = model
    self.lr = lr
    self.loss = nn.MSELoss()

  def training_step(self, batch):
    x, y = batch
    y_hat, _ = self.model.forward(x)
    y_hat = y_hat.view_as(y) 
    loss = nn.MSELoss()(y_hat, y)
    self.log("train_loss", loss, on_step=False, on_epoch=True)
    return {"loss": loss}

  def validation_step(self, batch):
    x, y = batch
    y_hat, _ = self.model.forward(x)
    y_hat = y_hat.view_as(y)
    loss = nn.MSELoss()(y_hat, y)
    self.log("val_loss", loss, on_step=False, on_epoch=True)
    return {"loss": loss}

  def test_step(self, batch):
    x, y = batch
    y_hat, _ = self.model.forward(x)
    y_hat = y_hat.view_as(y)
    loss = nn.MSELoss()(y_hat, y)
    self.log("test_loss", loss, prog_bar=True)
    return {"loss": loss}
    
  def configure_optimizers(self):
    return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [ ]:
out_features = 6 # Output
in_features = 561 # Input

In [ ]:
wiring = AutoNCP(training_configs["num_neurons"], out_features)

In [ ]:
ltc_model = LTC(in_features, wiring, batch_first=True)
learn = SequenceLearner(ltc_model, lr=training_configs["learning_rate"])

log_dir = f"logs"
logger = CSVLogger(log_dir, name=model_name)

trainer = pl.Trainer(
	log_every_n_steps=1,
	logger=logger,
	max_epochs=training_configs["max_epochs"],
	callbacks=[TimingCallback()],
	gradient_clip_val=1,  # Clip gradient to stabilize training
)

Training

In [ ]:
if has_validation:
	trainer.fit(learn, train_dataloader, val_dataloader)
else:
	trainer.fit(learn, train_dataloader)

Testing

In [ ]:
trainer.test(learn, test_dataloader)

Calculating the accuracy

In [ ]:
prediction_results = ltc_model(test_x)[0]
test_len = test_x.shape[0]

In [ ]:
hits = 0

for i in range(test_len):
  prediction = prediction_results[i].argmax()
  label_response = test_y[i].argmax()

  if prediction == label_response:
    hits += 1
    
print(f"Model's Accuracy On Test Data: {hits/test_x.shape[0]:.4f}")

Saving the trained model

In [ ]:
if not os.path.exists("models"):
	os.makedirs("models")

torch.save(ltc_model, f"models/{model_name}.pt")

Saving the configuration used

In [ ]:
config_file_path = os.path.join(logger.log_dir, "config.json")
with open(config_file_path, 'w') as config_file:
	json.dump(configs, config_file, indent=2)

Notification for finishing the training

In [ ]:
notification.notify(
	title="Training ended",
	message=f"The training of the model {model_name} with {training_configs['max_epochs']} epochs has been completed.",
	timeout=10
)